#### Web Scraping 
Es un método de extracción de datos que se basa en el proceso de recopilar y extraer datos provenientes de sitios web como texto, imágenes y enlaces mediante simulaciones de navegación web.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException
import pandas as pd
import numpy as np
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

#### Ofertas de empleo sobre ciencia de datos sin experiencia y del último mes

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument("--window-size=1300,700")

driver = webdriver.Chrome(options=option)
    
try: 
    driver.get("https://www.linkedin.com")
    
    job_search = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, '//a[@href="https://www.linkedin.com/jobs/search?trk=guest_homepage-basic_guest_nav_menu_jobs"]'))
    )
    job_search.click()
    
    button_quit = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, '//icon[@class="contextual-sign-in-modal__modal-dismiss-icon lazy-loaded" and @aria-hidden="true" and @aria-busy="false"]'))
    )
    button_quit.click()   
    
    input_workstation = driver.find_element(By.XPATH, '//input[@aria-controls="job-search-bar-keywords-typeahead-list"]')
    input_workstation.send_keys("Ciencia de datos")
    input_location = driver.find_element(By.XPATH, '//input[@aria-controls="job-search-bar-location-typeahead-list"]')
    input_location.clear()
    input_location.send_keys("Argentina")
    input_location.send_keys(Keys.RETURN)       
    
    driver.find_element(By.XPATH, '//button[@aria-label="Filtro «Fecha de publicación». Se ha aplicado el filtro «Cualquier momento». Al hacer clic en este botón, se muestran todas las opciones del filtro «Fecha de publicación»."]').click()                   
except ElementNotInteractableException: 
    button_quit = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, '//icon[@class="contextual-sign-in-modal__modal-dismiss-icon lazy-loaded" and @aria-hidden="true" and @aria-busy="false"]'))
    )
    button_quit.click() 
    
    driver.find_element(By.XPATH, '//button[@aria-label="Filtro «Fecha de publicación». Se ha aplicado el filtro «Cualquier momento». Al hacer clic en este botón, se muestran todas las opciones del filtro «Fecha de publicación»."]').click()
finally:    
    driver.find_element(By.XPATH, '//input[@id="f_TPR-1"]').click()
    driver.find_element(By.CLASS_NAME, 'filter__submit-button').click()
    
    workstation = driver.find_elements(By.CLASS_NAME, 'base-search-card__title')
    company = driver.find_elements(By.CLASS_NAME, 'hidden-nested-link')
    location = driver.find_elements(By.CLASS_NAME, 'job-search-card__location')
    time = driver.find_elements(By.CLASS_NAME, 'job-search-card__listdate')
    
    offers = driver.find_elements(By.XPATH, '//div[@class="base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card job-search-card--active"]')

data = {
    "workstation": [],
    "company": [],
    "location": [],
    "time": []
}

len_col = np.array([len(workstation),len(location),len(company),len(time)])
min_len_col = len_col.min()

def data_org(data, column, element):
    for e in element:
        data[column].append(e.text)
        if len(data[column]) == min_len_col:
            break
    return data

data_org(data, "workstation", workstation)
data_org(data, "company", company)
data_org(data, "location", location)
data_org(data, "time", time)

df = pd.DataFrame(data)
df

#### Estudios si o no
una vez obtenidas la ofertas de empleo, indagamos sobre ellas y descubrimos si contienen estudios universitarios como requisito

In [ ]:
studies = []
words_key = ("Estudiante","Licenciatura","Carrera","Ingeniería","Maestría")

for o in offers:
    link = o.find_element(By.XPATH, '//a[@class="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]"]').get_attribute("href")
    driver.get(link)
    
    button_quit = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, '//icon[@class="contextual-sign-in-modal__modal-dismiss-icon lazy-loaded" and @aria-hidden="true" and @aria-busy="false"]'))
    )
    button_quit.click() 
    
    driver.find_element(By.XPATH, '//icon[@class="show-more-less-html__button-icon show-more-less-button-icon lazy-loaded"]').click()
    
    info = driver.find_element(By.XPATH, '//div[@class="show-more-less-html__markup relative overflow-hidden"]')
    requests = info.find_elements(By.TAG_NAME, 'p')
    
    for r in requests:
        if any(w in r.text for w in words_key):
            studies.append("estudios universitarios requeridos")
        else:
            studies.append("estudios universitarios no requeridos")
            
    driver.quit()
            
df["studies"] = studies

#### Data Wrangling 
Es el proceso de limpiar, estructurar y transformar los datos disponibles con el fin de trabajar posteriormente con datos de alta calidad, bien organizados y con relevancia. Esta técnica se ha empleado en la mayoría de los proyectos anteriores si haber sido definida explicitamente y representa la mayor parte del trabajo de un científico de datos.

In [ ]:
print(df["location"].value_counts())

# Mapeo de datos
# dado que muchas ubicaciones son similares y sobreinforman, las reemplazamos y resumimos la iformación

data_mapping = {
    "Ciudad Autónoma de Buenos Aires, Provincia de Buenos Aires, Argentina": "Ciudad Autónoma de Buenos Aires",
    "Buenos Aires, Provincia de Buenos Aires, Argentina": "Provincia de Buenos Aires",
    "Buenos Aires y alrededores": "Provincia de Buenos Aires"
}

df["location"] = df["location"].apply(lambda x : data_mapping.get(x, x))
print("---------------------------------------------------------------------------")
print(df["location"].value_counts())

#### Dashboard que revela la ubicación y requisitos de estudios de las ofertas

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e8_body",children=[
    html.H1("Ofertas de trabajo en Linkedin", className="e8_title"),
        dcc.Dropdown(id="dropdown",className="e8_dropdown",
                    options = [
                        {"label":"Ubiación(Provincia, Comuna, Barrio)","value":"location"},
                        {"label":"Estudios universitarios","value":"studies"},
                    ],
                    value="location",
                    multi=False,
                    clearable=False),
    html.Div(className="e8_div",children=[
        dcc.Graph(id="graph_1",className="e8_graph",figure={}),
        dcc.Graph(id="graph_2",className="e8_graph",figure={})
    ])
])

@app.callback(
    [Output(component_id="graph_1",component_property="figure"),
    Output(component_id="graph_2",component_property="figure")],
    [Input(component_id="dropdown",component_property="value")]
)

def update_graph(slct_var):
    values_count = df[slct_var].value_counts().reset_index()
    
    barplot = px.bar(values_count, x=slct_var, y="count", title='Conteo de variables')
    piechart = px.pie(values_count, values='count', names=slct_var, title='Porciones de variables')
    piechart.update_layout(legend=dict(font=dict(size=6)))
    
    if slct_var == "location":
        barplot.update_layout(xaxis=dict(tickfont=dict(size=7)))
    
    return barplot, piechart
    
if __name__ == "__main__":
    app.run_server(debug=False)